In [3]:
#%%
import pandas as pd
import os
print(os.getcwd())
d = pd.read_csv('dest.csv', 
                header=None, 
                names=['age', 'age2', 'ed',
                       'exper', 'exper2', 'nearc2',
                       'nearc4', 'nearc4a', 'nearc4b',
                       'race', 'smsa', 'south', 'wage'])
print(d.columns.values)

C:\Users\Radim\Documents\GitHub\AdvEcox2
['age' 'age2' 'ed' 'exper' 'exper2' 'nearc2' 'nearc4' 'nearc4a' 'nearc4b'
 'race' 'smsa' 'south' 'wage']


C:\Users\Radim\Documents\GitHub\AdvEcox2
